In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import operator
import calendar

In [4]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
389,tt0955308,200000000,310669540,Robin Hood,Russell Crowe|Cate Blanchett|Mark Strong|Oscar...,Ridley Scott,"Rise and rise again, until lambs become lions.",When soldier Robin happens upon the dying Robe...,140,Action,Imagine Entertainment|Universal Pictures|Scott...,5/12/2010,6.1,2010
558,tt0179626,60000000,56359980,15 Minutes,Robert De Niro|Charlize Theron|Edward Burns|Ke...,John Herzfeld,America Likes to Watch,When Eastern European criminals Oleg and Emil ...,120,Action|Crime|Thriller,New Line Cinema|Industry Entertainment|Tribeca...,3/1/2001,5.7,2001
1752,tt0829459,16000000,18928871,A Mighty Heart,Angelina Jolie|Dan Futterman|Irrfan Khan|Archi...,Michael Winterbottom,One woman dared to keep hope alive,Based on Mariane Pearl's account of the terrif...,100,Drama|Thriller,Plan B Entertainment|Revolution Films|Paramoun...,1/1/2007,6.7,2007
1293,tt2473682,5000000,86362372,Paranormal Activity: The Marked Ones,Richard Cabral|Carlos Pratts|Eddie J. Fernande...,Christopher B. Landon,You're one of us now.,Seventeen-year-old Jesse has been hearing terr...,84,Horror|Thriller,Blumhouse Productions|Room 101|Solana Films,12/31/2013,5.3,2013
1380,tt0399146,32000000,60740827,A History of Violence,Viggo Mortensen|Maria Bello|Heidi Hayes|Ashton...,David Cronenberg,Tom Stall had the perfect life... until he bec...,An average family is thrust into the spotlight...,96,Drama|Thriller|Crime,New Line Cinema|DC Comics|BenderSpink,9/23/2005,6.7,2005


In [5]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [6]:
answers = {} # создадим словарь для ответов

# добавим новые колонки
data['profit'] = data['revenue'] - data['budget']
data['release_month'] = data.release_date.apply(lambda x: int(x.split('/')[0]))
data['title_len'] = data.original_title.apply(lambda x: len(x))
data['descr_len'] = data.overview.apply(lambda x: len(x.split(' ')))

# функция для определения имени фильма из выборки
def get_film_name(films_dataset):
    film = films_dataset[['original_title', 'imdb_id']].values[0]
    return f'{film[0]} ({film[1]})'

# функция для получения словаря с количеством
def get_counts_dict(dataset, column, separator='|'):
    counts_dict = {}
    for row in dataset[column].str.split(separator):
        for value in row:
            if value not in counts_dict:
                counts_dict[value] = 1
            else:
                counts_dict[value] += 1
    return counts_dict

# функция для получения топового значения
def get_top_value(prepared_dict):
    # сортируем подготовленный словарь, возвращаем только текст из первой строки
    return sorted(
        prepared_dict.items(),
        key=operator.itemgetter(1),
        reverse=True
    )[0][0]



# 1. У какого фильма из списка самый большой бюджет?

In [7]:
answers['1'] = get_film_name(
    data[data.budget == data.budget.max()]
)

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
answers['2'] = get_film_name(
    data[data.runtime == data.runtime.max()][['original_title', 'imdb_id']]
)

# 3. Какой из фильмов самый короткий (в минутах)?





In [9]:
answers['3'] = get_film_name(
    data[data.runtime == data.runtime.min()]
)

# 4. Какова средняя длительность фильмов?


In [10]:
answers['4'] = round(data.runtime.mean())

# 5. Каково медианное значение длительности фильмов? 

In [11]:
answers['5'] = data.runtime.median()

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [12]:
answers['6'] = get_film_name(
    data[data.profit == data.profit.max()]
)

# 7. Какой фильм самый убыточный? 

In [13]:
answers['7'] = get_film_name(
    data[data.profit == data.profit.min()]
)

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [14]:
answers['8'] = data[data.revenue > data.budget].count()[0]

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [15]:
part_data = data[data.release_year == 2008]
answers['9'] = get_film_name(
    part_data[part_data.revenue == part_data.revenue.max()]
)

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [16]:
part_data = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
answers['10'] = get_film_name(
    part_data[part_data.profit == part_data.profit.min()]
)

# 11. Какого жанра фильмов больше всего?

In [17]:
# вариант 1 - перебор всех вариантов и запись их в словарь
genres_count = get_counts_dict(data, 'genres')

answers['11'] = get_top_value(genres_count)

In [18]:
# вариант 2 - получить новую таблицу, в каждой строке которой будет по жанру
# В качетсве ответа возьмём самый повторяемый вариант

answer_11 = pd.DataFrame(
    {'genre': np.concatenate(data.genres.str.split('|').values)}
).describe().loc['top'][0]

# убедимся, что полученный ответ совпадает с первым вариантом
assert answer_11 == answers['11'], 'Alternate answer is not equal to base answer'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [19]:
# создадим словарь с жанрами, сохраняя в него массив с прибылями
genres_profit = {}

for x in data[['genres', 'profit']].values:
    genres = x[0]
    profit = x[1]
    for genre in genres.split('|'):
        if genre not in genres_profit:
            genres_profit[genre] = []
        genres_profit[genre].append(profit)

# посчитаем среднюю прибыль по жанрам
genres_profit = {x: (sum(genres_profit[x]) / len(genres_profit[x])) for x in genres_profit}

# ответ на "Фильмы какого жанра чаще всего становятся прибыльными"
answers['12'] = get_top_value(genres_profit)  # Fantasy

# ответ на "Какого жанра среди прибыльных фильмов больше всего"
genres_count = get_counts_dict(data[data.profit > 0], 'genres')
answers['12'] = get_top_value(genres_count)  # Drama

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [20]:
answers['13'] = data.groupby(['director']).revenue.sum().sort_values(ascending=False).index[0]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [21]:
directors = {}
answer_options = ['Ridley Scott', 'Guy Ritchie', 'Robert Rodriguez', 'Quentin Tarantino', 'Tony Scott']

for x in data[['genres', 'director']].values:
    genres = x[0]
    director = x[1]
    if 'Action' in x[0]:
      for d in answer_options:
        if d in x[1]:
          if d not in directors:
           directors[d] = 1
          else:
            directors[d] += 1

directors = sorted(
    directors.items(),
    key=operator.itemgetter(1),
    reverse=True
)

answers['14'] = directors[0][0]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [22]:
part_data = data[data.release_year == 2012]
most_revenue_actors = part_data[part_data.revenue == part_data.revenue.max()].cast.values[0].split('|')
answer_options = ['Nicolas Cage', 'Tom Hardy', 'Chris Hemsworth', 'Jim Sturgess', 'Emma Stone']
answers['15'] = [x for x in most_revenue_actors if x in answer_options][0]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов? (бюджет выше среднего)

In [23]:
actors = get_counts_dict(data[data.budget > data.budget.mean()], 'cast')
answers['16'] = get_top_value(actors)

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [24]:
genres = get_counts_dict(data[data['cast'].str.contains("Nicolas Cage")], 'genres')
answers['17'] = get_top_value(genres)

# 18. Самый убыточный фильм от Paramount Pictures

In [25]:
paramount_films = data[data['production_companies'].str.contains("Paramount Pictures")]
answers['18'] = get_film_name(
    paramount_films[paramount_films.profit == paramount_films.profit.min()]
)

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [26]:
sum_rev = data.pivot_table(values = 'revenue', index = 'release_year', fill_value=0, aggfunc = 'sum')
answers['19'] = sum_rev[sum_rev.revenue == sum_rev.revenue.max()].index[0]

# 20. Какой самый прибыльный год для студии Warner Bros?

In [27]:
warn_data = data[data['production_companies'].str.contains('Warner Bros')]
sum_rev = warn_data.pivot_table(values = 'revenue', index = 'release_year', fill_value=0, aggfunc = 'sum')
answers['20'] = sum_rev[sum_rev.revenue == sum_rev.revenue.max()].index[0]

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [28]:
month_pivot = data.pivot_table(columns = 'release_month', aggfunc = 'count').iloc[0]
answers['21'] = calendar.month_name[month_pivot.idxmax()]

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [29]:
answers['22'] = sum(
    data[
        data.release_month.isin([6, 7, 8])
    ].pivot_table(columns = 'release_month', aggfunc = 'count').iloc[0].values
)

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [30]:
winter_directors = data[(data['release_month'] <= 2) | (data['release_month'] == 12)]
winter_pivot = winter_directors.pivot_table(index='director', values='imdb_id', aggfunc = 'count')
answers['23'] = winter_pivot[winter_pivot['imdb_id'] == winter_pivot['imdb_id'].max()].index[0]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [31]:
answers['24'] = data[data['title_len'] == data['title_len'].max()].production_companies.values[0].split('|')[-1]

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [32]:
answers['25'] = data[data['descr_len'] == data['descr_len'].max()].production_companies.values[0].split('|')[-1]

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [33]:
best_rating = data.quantile(q=0.99, axis=0, numeric_only=True)['vote_average']
data[data['vote_average'] > best_rating].original_title
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [34]:
# вариант 1 - проходим по всем актёрам в фильмах, сопоставляя пары в словаре и считая кол-во совпадений
cast_rel = {}
for actors in data.cast.str.split('|'):
    for actor_1 in actors:
        for actor_2 in actors:
            if actor_1 == actor_2:
                continue
            rel = f'{actor_1} & {actor_2}'
            if rel not in cast_rel:
                cast_rel[rel] = 0
            cast_rel[rel] += 1

answers['27'] = get_top_value(cast_rel)  # Daniel Radcliffe & Rupert Grint

In [35]:
# вариант 2

data_cast = pd.DataFrame({
    'cast': sorted(data.cast.str.split('|').values)
})

data_cast['pairs'] = data_cast.cast.apply(lambda x: combinations(x, 2))

f = [item for sublist in data_cast['pairs'].values for item in sublist]

answer_27 = ' & '.join(Counter(f).most_common()[1][0])

# убедимся, что полученный ответ совпадает с первым вариантом
assert answer_27 == answers['27'], 'Alternate answer is not equal to base answer'

# Submission

In [36]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '2': 'Gods and Generals (tt0279111)',
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107.0,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)'}

In [37]:
# и убедиться что ни чего не пропустил)
len(answers)

27